ALL AGENTING PIPELINES LIKE REACT AND REFLEXTION AGENT

In [2]:
import requests
import datetime
import os
from groq import Groq
from dotenv import load_dotenv
import pickle
from datetime import datetime

load_dotenv()

True

In [3]:
with open('KG/new_graph.pkl', 'rb') as f:
    G = pickle.load(f)

### FUNCTIONS

In [26]:

def search_news(actor, recipient):
    if actor == "Congee":
        actor = 'Congress'
    if recipient == "Congee":
        recipient = 'Congress'
    query = f"{actor} and {recipient}"
    api_key = "fb8156f456be4094b9e88dd55b1d29fa" # Replace with your API key
    url = f"https://newsapi.org/v2/everything?q={query}&sortBy=publishedAt&apiKey={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        articles = response.json().get('articles', [])
        return [{"title": article['title'], "description": article['description'], "url": article['url']} for article in articles[:1]]
    else:
        print(f"Error fetching news articles: {response.status_code}, {response.text}")
        return []


In [16]:

def get_node_edge_connections(actor, recipient, date,graph=G):
    relevant_edges = []
    relevant_nodes = set()
    for u, v, data in graph.edges(data=True):
        if (actor in [u, v] or recipient in [u, v]) and data.get('relation') == 'event':
            event_date = graph.nodes[u].get('event_date', graph.nodes[v].get('event_date'))
            if event_date and datetime.datetime.strptime(event_date, "%Y-%m-%d") < datetime.datetime.strptime(date, "%Y-%m-%d"):
                relevant_edges.append((u, v, data))
                relevant_nodes.add(u)
                relevant_nodes.add(v)
    return relevant_nodes, relevant_edges


In [17]:

def print_node_attributes(actor, recipient, input_date,graph=G):
    # Convert input_date to datetime object for comparison
    input_date = datetime.strptime(input_date, "%Y-%m-%d")

    # Create a list to store the events that match the criteria
    matching_events = []

    # Iterate through all nodes in the graph
    for node in graph.nodes:
        # Check if the node has event_id attribute and is an event node
        if 'event_date' in graph.nodes[node]:
            # Get the event date and check if actor and recipient match
            event_date = datetime.strptime(graph.nodes[node]['event_date'], "%Y-%m-%d")
            
            # Find the actor and recipient neighbors
            actor_neighbors = list(graph.neighbors(node))
            recipient_neighbors = list(graph.neighbors(node))
            
            if actor in actor_neighbors and recipient in recipient_neighbors:
                # Calculate the time difference between input date and event date
                time_diff = abs((event_date - input_date).days)
                
                # Append the event along with time difference to the list
                matching_events.append((node, time_diff, graph.nodes[node]))

    # Sort the matching events based on time difference (closest dates first)
    matching_events.sort(key=lambda x: x[1])

    # Select the top 20 events
    top_20_events = matching_events[:5]

    # Return the node attributes for the top 20 events
    top_20_event_data = {event[0]: event[2] for event in top_20_events}
    return top_20_event_data


In [18]:
def calculate_event_type_frequency(actor, recipient, date,graph=G):
    event_counts = {}
    for u, v, data in graph.edges(data=True):
        if (actor in [u, v] or recipient in [u, v]):
            event_date = graph.nodes[u].get('event_date', graph.nodes[v].get('event_date'))
            if event_date and datetime.strptime(event_date, "%Y-%m-%d") < datetime.strptime(date, "%Y-%m-%d"):
                event_type = graph.nodes[u].get('event_type', graph.nodes[v].get('event_type'))
                if event_type:
                    event_counts[event_type] = event_counts.get(event_type, 0) + 1
    return event_counts


In [19]:
# Initialize Groq client
client = Groq(api_key=os.getenv("GROQ_API_KEY"))

def query_groq_for_prediction(actor, recipient, date, graph):
    # Step 1: Fetch news articles
    news_articles = search_news(actor, recipient)

    # Step 2: Extract nodes and edges
    nodes, edges = get_node_edge_connections(graph, actor, recipient, date)

    # Step 3: Output node attributes
    node_attributes = print_node_attributes(graph, actor,recipient, date)

    # Step 4: Calculate event type frequency
    event_frequencies = calculate_event_type_frequency(graph, actor, recipient, date)

    # Step 5: Create a structured prompt
    prompt = f"""
You are a knowledge graph analysis expert. Based on the following data, predict the most likely event type between '{actor}' and '{recipient}' on '{date}'.
Recent News Articles:
{news_articles}

*THIS IS KNOWLEDGE GRAPH DATA FOR THE {actor} and {recipient} relations/events
Event Frequency:
{event_frequencies}

Node Attributes:
{node_attributes}

Choose the most likely event type from the following list:
ACCUSE, THREATEN, COERCE, SANCTION, REQUEST, ASSAULT, RETREAT, CONCEDE, MOBILIZE, PROTEST, CONSULT, AID
"""
    print(prompt)
    try:
        chat_completion = client.chat.completions.create(
            messages=[{"role": "user", "content": prompt}],
            model="mixtral-8x7b-32768",
        )
        return chat_completion.choices[0].message.content
    except Exception as e:
        print(f"Error querying Groq API: {e}")
        return None

In [50]:
# Define the inputs
actor = "Bharatiya Janata Party"
recipient = "Congee"
date = "2024-06-25"

# Run the prediction
prediction = query_groq_for_prediction(actor, recipient, date, G)
print("Predicted Event Type:", prediction)


You are a knowledge graph analysis expert. Based on the following data, predict the most likely event type between 'Bharatiya Janata Party' and 'Congee' on '2024-06-25'.
Recent News Articles:
[{'title': "CNA Explains: What you need to know about India's 'one nation, one election' plan", 'description': "What would the proposal entail, and what are the benefits and drawbacks? CNA's Rohit Rajan breaks it down.", 'url': 'https://www.channelnewsasia.com/asia/india-one-nation-one-election-cna-explains-4813646'}]

*THIS IS KNOWLEDGE GRAPH DATA FOR THE Bharatiya Janata Party and Congee relations/events
Event Frequency:
{'PROTEST': 25, 'THREATEN': 21, 'ACCUSE': 149, 'REQUEST': 17, 'CONCEDE': 22, 'RETREAT': 22, 'MOBILIZE': 26, 'ASSAULT': 4, 'SANCTION': 23, 'COERCE': 10}

Node Attributes:
{'20240619-0830-ca901f31bcb4_ACCUSE': {'event_date': '2024-06-19', 'event_type': 'ACCUSE', 'event_intensity': -3.0, 'quad_code': 'VERBAL CONFLICT', 'contexts': 'election '}, '20240619-5134-c522fb30767d_MOBILIZE

### TOOLS AND AGENT

In [ ]:
from langchain.agents import initialize_agent, AgentType
from langchain.tools import Tool
from langchain.chat_models import ChatOpenAI
import requests
import os
import networkx as nx
import datetime
import json

# Define individual tools

def search_news(input_data):
    input_data = json.loads(input_data)
    actor = input_data["actor"]
    recipient = input_data["recipient"]
    query = f"{actor} and {recipient}"
    api_key = os.getenv("NEWS_API_KEY")  # Replace with your API key
    url = f"https://newsapi.org/v2/everything?q={query}&sortBy=publishedAt&apiKey={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        articles = response.json().get('articles', [])
        return [{"title": article['title'], "description": article['description'], "url": article['url']} for article in articles[:1]]
    else:
        return []

def get_node_edge_connections(input_data):
    input_data = json.loads(input_data)
    graph = input_data["graph"]
    actor = input_data["actor"]
    recipient = input_data["recipient"]
    date = input_data["date"]
    
    relevant_edges = []
    relevant_nodes = set()
    for u, v, data in graph.edges(data=True):
        if (actor in [u, v] or recipient in [u, v]) and data.get('relation') == 'event':
            event_date = graph.nodes[u].get('event_date', graph.nodes[v].get('event_date'))
            if event_date and datetime.datetime.strptime(event_date, "%Y-%m-%d") < datetime.datetime.strptime(date, "%Y-%m-%d"):
                relevant_edges.append((u, v, data))
                relevant_nodes.add(u)
                relevant_nodes.add(v)
    return {"nodes": list(relevant_nodes), "edges": relevant_edges}

def print_node_attributes(input_data):
    input_data = json.loads(input_data)
    graph = input_data["graph"]
    actor = input_data["actor"]
    recipient = input_data["recipient"]
    input_date = input_data["date"]
    
    input_date = datetime.datetime.strptime(input_date, "%Y-%m-%d")
    matching_events = []
    for node in graph.nodes:
        if 'event_date' in graph.nodes[node]:
            event_date = datetime.datetime.strptime(graph.nodes[node]['event_date'], "%Y-%m-%d")
            actor_neighbors = list(graph.neighbors(node))
            recipient_neighbors = list(graph.neighbors(node))
            if actor in actor_neighbors and recipient in recipient_neighbors:
                time_diff = abs((event_date - input_date).days)
                matching_events.append((node, time_diff, graph.nodes[node]))
    matching_events.sort(key=lambda x: x[1])
    top_5_events = matching_events[:5]
    return {event[0]: event[2] for event in top_5_events}

def calculate_event_type_frequency(input_data):
    input_data = json.loads(input_data)
    graph = input_data["graph"]
    actor = input_data["actor"]
    recipient = input_data["recipient"]
    date = input_data["date"]
    
    event_counts = {}
    for u, v, data in graph.edges(data=True):
        if actor in [u, v] or recipient in [u, v]:
            event_date = graph.nodes[u].get('event_date', graph.nodes[v].get('event_date'))
            if event_date and datetime.datetime.strptime(event_date, "%Y-%m-%d") < datetime.datetime.strptime(date, "%Y-%m-%d"):
                event_type = graph.nodes[u].get('event_type', graph.nodes[v].get('event_type'))
                if event_type:
                    event_counts[event_type] = event_counts.get(event_type, 0) + 1
    return event_counts

def query_groq_for_prediction(input_data):
    input_data = json.loads(input_data)
    actor = input_data["actor"]
    recipient = input_data["recipient"]
    date = input_data["date"]
    graph = input_data["graph"]
    
    news_articles = search_news(input_data)
    nodes_edges = get_node_edge_connections(input_data)
    node_attributes = print_node_attributes(input_data)
    event_frequencies = calculate_event_type_frequency(input_data)
    
    prompt = f"""
    You are a knowledge graph analysis expert. Predict the most likely event type between '{actor}' and '{recipient}' on '{date}'.
    
    Recent News Articles:
    {news_articles}
    
    Event Frequency:
    {event_frequencies}
    
    Node Attributes:
    {node_attributes}
    
    Choose from: ACCUSE, THREATEN, COERCE, SANCTION, REQUEST, ASSAULT, RETREAT, CONCEDE, MOBILIZE, PROTEST, CONSULT, AID.
    """
    llm = ChatOpenAI(model="gpt-4", temperature=0.7)
    return llm.predict(prompt)

# Define tools for the agent
tools = [
    Tool(name="Search News", func=search_news, description="Fetch latest news articles about an actor and recipient."),
    Tool(name="Get Node Edge Connections", func=get_node_edge_connections, description="Extract node and edge data from a knowledge graph."),
    Tool(name="Print Node Attributes", func=print_node_attributes, description="Retrieve event details related to actor and recipient."),
    Tool(name="Calculate Event Frequency", func=calculate_event_type_frequency, description="Determine historical event frequency."),
    Tool(name="Predict Event", func=query_groq_for_prediction, description="Predict the most probable event type using knowledge graph and news data.")
]

# Initialize the React agent
llm = ChatOpenAI(model="gpt-4", temperature=0.7)
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

def run_event_prediction(actor, recipient, date, graph):
    input_data = {
        "actor": actor,
        "recipient": recipient,
        "date": date,
        "graph": graph
    }
    return agent.run({"input": input_data})

output = run_event_prediction("Bharatiya Janata Party", "Congee", "2024-06-25", G)
print(output)


In [10]:
#https://github.com/alejandro-ao/agents-no-framework/blob/master/agent-w-loop.ipynb

from dotenv import load_dotenv
import os
from groq import Groq


load_dotenv()

True

In [11]:
class Agent:
    def __init__(self, client: Groq, system: str = "") -> None:
        self.client = client
        self.system = system
        self.messages: list = []
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message=""):
        if message:
            self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        completion = client.chat.completions.create(
            model="llama3-70b-8192", messages=self.messages
        )
        return completion.choices[0].message.content

In [27]:
class Agent:
    def __init__(self, client: Groq, system: str = "") -> None:
        self.client = client
        self.system = system
        self.messages: list = []
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message=""):
        if message:
            self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        completion = client.chat.completions.create(
            model="llama3-70b-8192", messages=self.messages
        )
        return completion.choices[0].message.content

system_prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop, you output an Answer.
Use Thought to describe your reasoning process for the question you have been asked.
Use Action to run one of the actions available to you, then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

search_news:
e.g. search_news: {"actor": "ActorName", "recipient": "RecipientName"}
Fetches the latest news articles about an actor and recipient, returning a list of articles with titles, descriptions, and URLs.

get_node_edge_connections:
e.g. get_node_edge_connections: {"actor": "ActorName", "recipient": "RecipientName", "date": "YYYY-MM-DD"}
Extracts relevant node and edge data from a knowledge graph, returning connected nodes and edges for the actor and recipient up to the given date.

print_node_attributes:
e.g. print_node_attributes: {"actor": "ActorName", "recipient": "RecipientName", "date": "YYYY-MM-DD"}
Retrieves event details related to an actor and recipient, prioritizing events nearest to the given date.

calculate_event_type_frequency:
e.g. calculate_event_type_frequency: {"actor": "ActorName", "recipient": "RecipientName", "date": "YYYY-MM-DD}
Determines historical event type frequencies involving an actor and recipient up to the given date.

Example session:

Question: What is the most likely event between Bharatiya Janata Party and Congee on 2024-06-25?
Thought: I need to analyze news articles, graph connections, and event attributes to predict the most likely event type.
Action: search_news: {"actor": "Bharatiya Janata Party", "recipient": "Congee"}
PAUSE

You will be called again with this:

Observation: [{"title": "Example News", "description": "Example Description", "url": "example.com"}]

Thought: I need to extract node and edge connections from the knowledge graph.
Action: get_node_edge_connections: {"actor": "Bharatiya Janata Party", "recipient": "Congee", "date": "2024-06-25"}
PAUSE

You will be called again with this:

Observation: {"nodes": ["Node1", "Node2"], "edges": [{"from": "Node1", "to": "Node2", "relation": "event"}]}

Thought: I need to retrieve node attributes for events close to the given date.
Action: print_node_attributes: {"actor": "Bharatiya Janata Party", "recipient": "Congee", "date": "2024-06-25"}
PAUSE

You will be called again with this:

Observation: {"Node1": {"event_date": "2024-06-20", "event_type": "CONSULT"}}

Thought: I need to determine the historical event type frequencies.
Action: calculate_event_type_frequency: {"actor": "Bharatiya Janata Party", "recipient": "Congee", "date": "2024-06-25"}
PAUSE

You will be called again with this:

Observation: {"CONSULT": 3, "REQUEST": 2}

Thought: I have all the necessary information to predict the most likely event type.
Thought: Based on the observations, the most frequent event type is CONSULT. This aligns with the recent events and historical frequencies.

Answer: The most likely event type between Bharatiya Janata Party and Congee on 2024-06-25 is CONSULT.

Now it's your turn:
""".strip()


In [28]:
import re
import time

client = Groq(api_key=os.environ.get("GROQ_API_KEY"))

def loop(max_iterations=10, query: str = ""):

    agent = Agent(client=client, system=system_prompt)

    tools = ["search_news", "get_node_edge_connections","print_node_attributes","calculate_event_type_frequency"]

    next_prompt = query

    i = 0
  
    while i < max_iterations:
        i += 1
        result = agent(next_prompt)
        print(result)

        if "PAUSE" in result and "Action" in result:
            action = re.findall(r"Action: ([a-z_]+): (.+)", result, re.IGNORECASE)
            chosen_tool = action[0][0]
            raw_args = action[0][1]

            if chosen_tool in tools:
                try:
                    # Parse arguments safely as JSON or fallback to raw string if parsing fails
                    try:
                        args = json.loads(raw_args)
                    except json.JSONDecodeError:
                        args = raw_args  # Use as-is if not a JSON object

                    # Handle tools with multiple arguments (dict unpacking for functions)
                    if isinstance(args, dict):
                        result_tool = eval(f"{chosen_tool}(**args)")
                    else:
                        result_tool = eval(f"{chosen_tool}('{args}')")

                    next_prompt = f"Observation: {result_tool}"
                except Exception as e:
                    next_prompt = f"Observation: Error - {str(e)}"
            else:
                next_prompt = "Observation: Tool not found"

            print(next_prompt)
            time.sleep(0.5)
            continue

        if "Answer" in result:
            break


loop(query="What is the most likely relation between 'Bharatiya Janata Party' and 'Congee' on 2024-06-25 \
       Choose from: ACCUSE, THREATEN, COERCE, SANCTION, REQUEST, ASSAULT, RETREAT, CONCEDE, MOBILIZE, PROTEST, CONSULT, AID.")

InternalServerError: Error code: 503 - {'error': {'message': 'Service Unavailable', 'type': 'internal_server_error'}}